# 생활인구 데이터 전처리

이 노트북은 `population_data.csv` 파일을 전처리하여 분기별 평균 생활인구 데이터를 생성합니다.

In [1]:
import pandas as pd
import numpy as np

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 import 완료")

✅ 라이브러리 import 완료


## Step 1: 데이터 로드

In [2]:
print("📂 데이터 로딩 중...")
df = pd.read_csv('data/population/population_data.csv', encoding='utf-8-sig')
print(f"✅ 원본 데이터: {len(df):,}개 행")
print(f"✅ 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"\n📋 데이터 구조:")
print(df.info())

📂 데이터 로딩 중...
✅ 원본 데이터: 18,591,552개 행
✅ 메모리 사용량: 2155.7 MB

📋 데이터 구조:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18591552 entries, 0 to 18591551
Data columns (total 7 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   기준일ID          int64 
 1   시간대구분          int64 
 2   행정동코드          int64 
 3   행정동명           object
 4   총생활인구수         int64 
 5   남자 20대-60대 인구  int64 
 6   여자 20대-60대 인구  int64 
dtypes: int64(6), object(1)
memory usage: 992.9+ MB
None


## Step 2: 기준일ID를 날짜로 변환

In [3]:
print("📅 날짜 처리 중...")

# 20200101 → datetime으로 변환
df['기준일자'] = pd.to_datetime(df['기준일ID'].astype(str), format='%Y%m%d')

# 연도, 분기 추출
df['연도'] = df['기준일자'].dt.year
df['분기'] = df['기준일자'].dt.quarter
df['월'] = df['기준일자'].dt.month

print(f"✅ 연도 범위: {df['연도'].min()} ~ {df['연도'].max()}")
print(f"✅ 분기: {sorted(df['분기'].unique())}")
print(f"✅ 데이터 연월: {df['연도'].min()}년 {df['월'].min()}월 ~ {df['연도'].max()}년 {df['월'].max()}월")
print(f"\n샘플 데이터:")
print(df[['기준일ID', '기준일자', '연도', '분기', '월']].head(10))

📅 날짜 처리 중...
✅ 연도 범위: 2020 ~ 2024
✅ 분기: [np.int32(1), np.int32(2), np.int32(3), np.int32(4)]
✅ 데이터 연월: 2020년 1월 ~ 2024년 12월

샘플 데이터:
      기준일ID       기준일자    연도  분기  월
0  20200101 2020-01-01  2020   1  1
1  20200101 2020-01-01  2020   1  1
2  20200101 2020-01-01  2020   1  1
3  20200101 2020-01-01  2020   1  1
4  20200101 2020-01-01  2020   1  1
5  20200101 2020-01-01  2020   1  1
6  20200101 2020-01-01  2020   1  1
7  20200101 2020-01-01  2020   1  1
8  20200101 2020-01-01  2020   1  1
9  20200101 2020-01-01  2020   1  1


## Step 3: 시간대 매핑

processed_data.csv와 동일한 시간대 구분으로 매핑:
- 0-5 → 00-06
- 6-10 → 06-11
- 11-13 → 11-14
- 14-16 → 14-17
- 17-20 → 17-21
- 21-23 → 21-24

In [4]:
print("⏰ 시간대 매핑 중...")

# 시간대 매핑 함수
def map_timeslot(hour):
    if 0 <= hour <= 5:
        return 'TMZON_00_06'
    elif 6 <= hour <= 10:
        return 'TMZON_06_11'
    elif 11 <= hour <= 13:
        return 'TMZON_11_14'
    elif 14 <= hour <= 16:
        return 'TMZON_14_17'
    elif 17 <= hour <= 20:
        return 'TMZON_17_21'
    elif 21 <= hour <= 23:
        return 'TMZON_21_24'
    else:
        return None

# 시간대구분 칼럼에 매핑 적용
df['시간대명'] = df['시간대구분'].apply(map_timeslot)

print(f"✅ 시간대 매핑 결과:")
print(df.groupby('시간대명').size().sort_index())
print(f"\n샘플:")
print(df[['시간대구분', '시간대명']].drop_duplicates().sort_values('시간대구분'))

⏰ 시간대 매핑 중...
✅ 시간대 매핑 결과:
시간대명
TMZON_00_06    4647888
TMZON_06_11    3873240
TMZON_11_14    2323944
TMZON_14_17    2323944
TMZON_17_21    3098592
TMZON_21_24    2323944
dtype: int64

샘플:
      시간대구분         시간대명
0         0  TMZON_00_06
424       1  TMZON_00_06
848       2  TMZON_00_06
1272      3  TMZON_00_06
1696      4  TMZON_00_06
2120      5  TMZON_00_06
2544      6  TMZON_06_11
2968      7  TMZON_06_11
3392      8  TMZON_06_11
3816      9  TMZON_06_11
4240     10  TMZON_06_11
4664     11  TMZON_11_14
5088     12  TMZON_11_14
5512     13  TMZON_11_14
5936     14  TMZON_14_17
6360     15  TMZON_14_17
6784     16  TMZON_14_17
7208     17  TMZON_17_21
7632     18  TMZON_17_21
8056     19  TMZON_17_21
8480     20  TMZON_17_21
8904     21  TMZON_21_24
9328     22  TMZON_21_24
9752     23  TMZON_21_24


## Step 4: 그룹화 및 평균 계산

행정동별, 연도별, 분기별, 시간대별로 그룹화하여 평균 생활인구 계산

In [5]:
print("📊 분기별 평균 계산 중...")

# 그룹별로 묶기
grouped = df.groupby([
    '행정동코드',
    '행정동명',
    '연도',
    '분기',
    'TIMESLOT'
])

# 평균 계산 (사용 가능한 칼럼만 사용)
평균_df = grouped.agg({
    '총생활인구수': 'mean',
    '남자 20대-60대 인구': 'mean',
    '여자 20대-60대 인구': 'mean'
}).reset_index()

# 칼럼명 변경
평균_df.columns = [
    '행정동코드', '행정동명', '연도', '분기', '시간대명',
    '평균생활인구수', '남자평균', '여자평균'
]

# 정수로 반올림
평균_df['평균생활인구수'] = 평균_df['평균생활인구수'].round(0).astype(int)
평균_df['남자평균'] = 평균_df['남자평균'].round(0).astype(int)
평균_df['여자평균'] = 평균_df['여자평균'].round(0).astype(int)

print(f"✅ 그룹화 후 데이터: {len(평균_df):,}개 행")
print(f"✅ 원본 대비 압축률: {len(df) / len(평균_df):.1f}배")
print(f"\n샘플 데이터:")
print(평균_df.head(10))

📊 분기별 평균 계산 중...


KeyError: 'TIMESLOT'

## Step 5: 피벗 테이블로 시간대별 칼럼 만들기

시간대를 개별 칼럼으로 펼쳐서 wide 형식으로 변환

In [ ]:
print("🔄 피벗 테이블 생성 중...")

# 시간대를 칼럼으로 펼치기 - 총 생활인구
pivot_df = 평균_df.pivot_table(
    index=['행정동코드', '행정동명', '연도', '분기'],
    columns='시간대명',
    values='평균생활인구수',
    aggfunc='mean',
    fill_value=0
).reset_index()

# 칼럼명 정리
pivot_df.columns.name = None
pivot_df.columns = [
    '행정동코드', '행정동명', '연도', '분기',
    '생활인구_TMZON_00_06_평균', '생활인구_TMZON_06_11_평균',
    '생활인구_TMZON_11_14_평균', '생활인구_TMZON_14_17_평균',
    '생활인구_TMZON_17_21_평균', '생활인구_TMZON_21_24_평균'
]

print(f"✅ Pivot 완료")
print(f"   행: {len(pivot_df):,}")
print(f"   칼럼: {pivot_df.shape[1]}")
print(f"\n=== Pivot 데이터 미리보기 ===")
print(pivot_df.head(10))
print(f"\n시간대별 칼럼:")
timeslot_cols = [col for col in pivot_df.columns if 'TMZON' in col]
for col in timeslot_cols:
    print(f"  - {col}")

## Step 6: 성별/연령대 데이터 추가

In [ ]:
print("👥 성별 데이터 추가 중...")

# 성별 평균도 피벗 (20-60대)
성별_pivot = 평균_df.groupby(['행정동코드', '행정동명', '연도', '분기']).agg({
    '남자평균': 'mean',
    '여자평균': 'mean'
}).reset_index()

# 정수로 변환
성별_pivot['남자평균'] = 성별_pivot['남자평균'].round(0).astype(int)
성별_pivot['여자평균'] = 성별_pivot['여자평균'].round(0).astype(int)

# 메인 데이터와 병합 (pivot_combined 사용)
최종_df = pivot_combined.merge(
    성별_pivot,
    on=['행정동코드', '행정동명', '연도', '분기'],
    how='left'
)

최종_df.rename(columns={
    '남자평균': '남성_20_60대_평균',
    '여자평균': '여성_20_60대_평균'
}, inplace=True)

print(f"✅ 성별 데이터 추가 완료")
print(f"\n최종 칼럼 목록 (총 {len(최종_df.columns)}개):")

# 칼럼 그룹별로 정리하여 출력
기본_칼럼 = [col for col in 최종_df.columns if col in ['행정동코드', '행정동명', '연도', '분기']]
생활인구_칼럼 = [col for col in 최종_df.columns if col.startswith('생활인구_')]
남성_칼럼 = [col for col in 최종_df.columns if col.startswith('남성_TMZON')]
여성_칼럼 = [col for col in 최종_df.columns if col.startswith('여성_TMZON')]
성별_20_60_칼럼 = [col for col in 최종_df.columns if '20_60대' in col]

print(f"\n📋 기본 정보 ({len(기본_칼럼)}개):")
for i, col in enumerate(기본_칼럼, 1):
    print(f"  {i}. {col}")

print(f"\n👥 시간대별 총 생활인구 ({len(생활인구_칼럼)}개):")
for i, col in enumerate(생활인구_칼럼, 1):
    print(f"  {i}. {col}")

print(f"\n♂️  시간대별 남성 인구 ({len(남성_칼럼)}개):")
for i, col in enumerate(남성_칼럼, 1):
    print(f"  {i}. {col}")

print(f"\n♀️  시간대별 여성 인구 ({len(여성_칼럼)}개):")
for i, col in enumerate(여성_칼럼, 1):
    print(f"  {i}. {col}")

print(f"\n👨👩 성별 20-60대 ({len(성별_20_60_칼럼)}개):")
for i, col in enumerate(성별_20_60_칼럼, 1):
    print(f"  {i}. {col}")

print(f"\n샘플 데이터:")
print(최종_df.head(3))

## Step 7: 총 생활인구 및 경제활동인구 계산

In [ ]:
print("➕ 총 생활인구 계산 중...")

# 시간대별 평균의 평균 계산
time_avg_cols = [col for col in 최종_df.columns if '생활인구_TMZON' in col and '평균' in col]
최종_df['총생활인구_평균'] = 최종_df[time_avg_cols].mean(axis=1).round(0).astype(int)

# 경제활동가능인구 (20-60대 남녀 합계)
최종_df['경제활동인구_평균'] = (최종_df['남성_20_60대_평균'] + 최종_df['여성_20_60대_평균']).astype(int)

print(f"✅ 계산 완료")
print(f"\n기초 통계:")
print(최종_df[['총생활인구_평균', '경제활동인구_평균', '남성_20_60대_평균', '여성_20_60대_평균']].describe())

➕ 총 생활인구 계산 중...
✅ 계산 완료

기초 통계:
            총생활인구_평균      경제활동인구_평균  남성_20_60대_평균  여성_20_60대_평균
count    8480.000000    8480.000000   8480.000000   8480.000000
mean    24453.132193   18123.419222   8686.830307   9436.588915
std     12772.461002   10463.065906   5386.837809   5173.033948
min      2716.000000    2050.000000   1061.000000    989.000000
25%     15983.250000   11385.500000   5316.750000   6014.750000
50%     21527.000000   15687.000000   7430.500000   8237.500000
75%     30341.000000   22219.750000  10519.000000  11677.250000
max    115771.000000  103187.000000  54689.000000  48801.000000


## Step 8: 결측치 처리

In [ ]:
print("🔍 결측치 확인 중...")

missing_counts = 최종_df.isnull().sum()
print(f"결측치 현황:")
print(missing_counts[missing_counts > 0])

if 최종_df.isnull().sum().sum() > 0:
    print(f"\n⚠️  결측치 발견! 0으로 채웁니다.")
    최종_df.fillna(0, inplace=True)
    print(f"✅ 결측치 처리 완료")
else:
    print(f"\n✅ 결측치 없음")

🔍 결측치 확인 중...
결측치 현황:
Series([], dtype: int64)

✅ 결측치 없음


## Step 9: 데이터 검증

In [ ]:
print("✅ 데이터 검증 중...")

print(f"\n📋 최종 데이터 요약:")
print(f"   총 행 수: {len(최종_df):,}개")
print(f"   총 칼럼 수: {len(최종_df.columns)}개")
print(f"   행정동 수: {최종_df['행정동명'].nunique()}개")
print(f"   연도 범위: {최종_df['연도'].min()} ~ {최종_df['연도'].max()}")
print(f"   분기: {sorted(최종_df['분기'].unique())}")

print(f"\n📊 샘플 데이터 (상위 10개):")
print(최종_df.head(10))

print(f"\n📈 기초 통계:")
print(최종_df.describe())

✅ 데이터 검증 중...

📋 최종 데이터 요약:
   총 행 수: 8,480개
   총 칼럼 수: 14개
   행정동 수: 423개
   연도 범위: 2020 ~ 2024
   분기: [np.int32(1), np.int32(2), np.int32(3), np.int32(4)]

📊 샘플 데이터 (상위 10개):
      행정동코드   행정동명    연도  분기  생활인구_TMZON_00_06_평균  생활인구_TMZON_06_11_평균  \
0  11110515  청운효자동  2020   1                14620                16081   
1  11110515  청운효자동  2020   2                14717                16401   
2  11110515  청운효자동  2020   3                14615                16378   
3  11110515  청운효자동  2020   4                14386                16510   
4  11110515  청운효자동  2021   1                14650                16359   
5  11110515  청운효자동  2021   2                14667                17091   
6  11110515  청운효자동  2021   3                14888                16810   
7  11110515  청운효자동  2021   4                15796                18503   
8  11110515  청운효자동  2022   1                15417                17560   
9  11110515  청운효자동  2022   2                15601                20257   

   생활인구_

## Step 10: 데이터 저장

In [ ]:
print("💾 데이터 저장 중...")

# CSV 파일로 저장
output_path = 'data/population/processed_people_data.csv'
최종_df.to_csv(output_path, index=False, encoding='utf-8-sig')

file_size = pd.read_csv(output_path).memory_usage(deep=True).sum() / 1024**2
print(f"✅ 완료! '{output_path}' 파일로 저장되었습니다.")
print(f"   파일 크기: {file_size:.1f} MB")
print(f"   원본 대비 압축률: {992.9 / file_size:.1f}배")

💾 데이터 저장 중...
✅ 완료! 'data/population/생활인구_전처리완료.csv' 파일로 저장되었습니다.
   파일 크기: 1.4 MB
   원본 대비 압축률: 691.3배


## 추가: 샘플 조회 함수

In [ ]:
def 샘플조회(행정동명, 연도=2020, 분기=1):
    """
    특정 행정동의 데이터 조회

    Parameters:
    -----------
    행정동명 : str
        조회할 행정동 이름
    연도 : int
        조회할 연도 (기본값: 2020)
    분기 : int
        조회할 분기 (기본값: 1)
    """
    결과 = 최종_df[
        (최종_df['행정동명'] == 행정동명) &
        (최종_df['연도'] == 연도) &
        (최종_df['분기'] == 분기)
    ]

    if len(결과) == 0:
        print(f"❌ '{행정동명}' ({연도}년 {분기}분기) 데이터를 찾을 수 없습니다.")
        print(f"\n사용 가능한 행정동: {sorted(최종_df['행정동명'].unique())[:10]}...")
    else:
        print(f"\n📍 {행정동명} ({연도}년 {분기}분기)")
        print("=" * 80)

        # 시간대별 생활인구
        print(f"\n⏰ 시간대별 평균 생활인구:")
        for col in 결과.columns:
            if '생활인구_TMZON' in col and '평균' in col:
                time_label = col.replace('생활인구_', '').replace('_평균', '')
                print(f"   {time_label}: {결과[col].values[0]:,}명")

        # 성별 및 총계
        print(f"\n👥 성별 및 총계:")
        print(f"   총생활인구 평균: {결과['총생활인구_평균'].values[0]:,}명")
        print(f"   경제활동인구 평균: {결과['경제활동인구_평균'].values[0]:,}명")
        print(f"   남성 20-60대 평균: {결과['남성_20_60대_평균'].values[0]:,}명")
        print(f"   여성 20-60대 평균: {결과['여성_20_60대_평균'].values[0]:,}명")

        return 결과

# 테스트 - 첫 번째 행정동으로 조회
첫번째_행정동 = 최종_df['행정동명'].iloc[0]
첫번째_연도 = 최종_df['연도'].iloc[0]
첫번째_분기 = 최종_df['분기'].iloc[0]

print(f"\n🔍 샘플 조회 테스트:")
샘플조회(첫번째_행정동, 첫번째_연도, 첫번째_분기)


🔍 샘플 조회 테스트:

📍 청운효자동 (2020년 1분기)

⏰ 시간대별 평균 생활인구:
   TMZON_00_06: 14,620명
   TMZON_06_11: 16,081명
   TMZON_11_14: 17,056명
   TMZON_14_17: 17,031명
   TMZON_17_21: 15,040명
   TMZON_21_24: 14,259명

👥 성별 및 총계:
   총생활인구 평균: 15,681명
   경제활동인구 평균: 11,709명
   남성 20-60대 평균: 5,863명
   여성 20-60대 평균: 5,846명


,행정동코드,행정동명,연도,분기,생활인구_TMZON_00_06_평균,생활인구_TMZON_06_11_평균,생활인구_TMZON_11_14_평균,생활인구_TMZON_14_17_평균,생활인구_TMZON_17_21_평균,생활인구_TMZON_21_24_평균,남성_20_60대_평균,여성_20_60대_평균,총생활인구_평균,경제활동인구_평균
0,11110515,청운효자동,2020,1,14620,16081,17056,17031,15040,14259,5863,5846,15681,11709


## 📊 요약

### 전처리 완료 항목:
1. ✅ 데이터 로드 (population_data.csv)
2. ✅ 날짜 변환 (기준일ID → 연도, 분기, 월)
3. ✅ 시간대 매핑 (0-23시 → 6개 시간대 그룹)
4. ✅ 행정동별, 연도별, 분기별, 시간대별 평균 계산
5. ✅ Wide 형식으로 변환 (시간대를 개별 칼럼으로)
6. ✅ 성별 데이터 추가 (남성/여성 20-60대)
7. ✅ 총 생활인구 및 경제활동인구 계산
8. ✅ 결측치 처리
9. ✅ 데이터 저장

### 최종 데이터 구조:
- **행정동코드, 행정동명**: 행정동 정보
- **연도, 분기**: 시간 정보
- **생활인구_TMZON_XX_평균**: 시간대별 평균 생활인구 (6개 칼럼)
- **총생활인구_평균**: 시간대별 평균의 평균
- **경제활동인구_평균**: 20-60대 남녀 합계
- **남성_20_60대_평균, 여성_20_60대_평균**: 성별 경제활동인구

### 다음 단계:
- 매출 데이터(processed_data.csv)와 병합하여 상관관계 분석
- 시간대별, 연령대별 매출과 생활인구의 관계 분석
- 행정동별 매출 예측 모델 개발